# Datenauswertung Grossbaustelle Bahnausbau in Bern, Wylerfeld

In diesem Jupyter-Notebook analysieren wir die Inklinometermessungen im Block 52, Baufeld B:

*   **I-52N**: Spundwand Nord, Kote Rohrmund 550.036 m ü.M.
*   **I-52S**: Spundwand Süd, Kote Rohrmund 550.247 m ü.M.

Die Inklinometerkonstante beträgt in beiden Fällen 100'000.

**Input:**
*   eine csv-Datei 'df_inclino_I-52S_I-52N.csv' mit den Rohdaten der beiden Inklinometer. Diese Datei wurde zum Teil von Hand, zum Teil mit Hilfe eines Python-Codes erstellt (nicht verfügbar auf Google Colab). Am 21.08.2020 wurde nur der Inklino I-52S gemessen. Diese Messung wird manuell aus der Datenbank entfernt, weil sie zu Schwierigkeiten in der Auswertung führt. Die Library "inclinometers" geht nämlich davon aus, dass beide Inklinometer am gleichen Tag beprobt wurden.

**Output:**
*   Listeneintrag
*   Listeneintrag







In [1]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

# working directory: drive/MyDrive/spundsteif
%cd '/content/drive/MyDrive/spundsteif'

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/spundsteif


# Libraries importieren

In [2]:
import pandas as pd
import numpy
import io
import csv
from datetime import datetime, date

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go

from spundsteif import inclinometers # we need also our own libraries

# Daten importieren

## Lesen der Inklinometerdaten

In [3]:
df = pd.read_csv('./data/df_inclino_I-52S_I-52N_neu.csv', delimiter=';', index_col=[0,1,2], header=[0,1])
df.index.set_names(['date', 'tube', 'depth'], inplace=True)
df.columns.rename(['direction', 'info value'], inplace=True)
df

direction                 A           B      
info value                +     -     +     -
date       tube  depth                       
06.03.2020 I-52N 0.5    419  -212 -1592  1339
                 1.0    402  -183 -1657  1389
                 1.5    338  -124 -1755  1525
                 2.0    316  -125 -1724  1391
                 2.5    440  -235 -1347  1052
...                     ...   ...   ...   ...
01.09.2020 I-52S 14.0  -292   498   247  -427
                 14.5  -678   879   204  -392
                 15.0  -905  1113   187  -393
                 15.5  -695   907   523  -696
                 16.0     8   202  1101 -1332

[3055 rows x 4 columns]

In [4]:
# questo codice funziona per accedere ai dati
# df.loc[('10.03.2020', 'I-52N'), :]

In [5]:
# How to set the indexes to a specific data type???
# attenzione, qui gli indici sono ancora nell'ordine sbagliato...

# tube = str
# date = datetime
# depth = float

# This does not work!
# df.Index(['tube', 'date', 'depth'], dtype=['str', 'datetime', 'float'])

# STACK OVERFLOW
# df.index = pd.to_datetime(df.index)

# df.index.names


In [ ]:
# The inclinometers library requires that each inclinometer is sampled on the same date.
# Let's check if this is the case with I-52N and I-52S.

dates_south = df.xs('I-52S', level='tube').xs(0.5, level='depth').index
dates_north = df.xs('I-52N', level='tube').xs(0.5, level='depth').index

dates_not_in_common_A = [i for i in dates_south if i not in dates_north]
print(dates_not_in_common_A)

dates_not_in_common_B = [i for i in dates_north if i not in dates_south]
print(dates_not_in_common_B)

# Yes, the dates correspond.

In [ ]:
# Let's use the functions from our inclinometers library.
# compute rotations and deformations
df = inclinometers.compute_average_A_B(df)
df = inclinometers.compute_rotations(df, konstant_inklino = 100000)

# Add relative deformations
df = inclinometers.column_integration(df)
#df = inclinometers.column_deformee_relative(df, '06.03.2020')

df

In [ ]:
# this is the code of the function <<< inclinometers.__deformee__(list_incl) >>>

In [ ]:
# this is the code of the function <<< inclinometers.column_integration(df) >>>

In [ ]:
# this is the code of the function <<< get_zero_measure_date(dates, start_date) >>>

# Given a start date returns the date of the measure closest in the past.
# dates: list of sorted datetetime.dates
# start_date: datetime.date

start_date = '06.03.2020'
elts_index = df.index.levels
dates, tubes = elts_index[0], elts_index[1]

print(dates)
print(dates[0])
print(type(dates[0]))

null_date = min(dates)

for date in dates:
  if date <= start_date:
    null_date = date

null_date

In [ ]:
# this is the code of the function <<< column_deformee_relative(df, start_date) >>>


# Add a column "relative deformations". The reference value is the start_date. All measures before this date are deleted.


# Pour s'assurer que les hauteurs coincident avec les valeurs de defo ajoutées
# et évite les alertes de performance
df = df.sort_index(axis=0)

idx = pd.IndexSlice

elts_index = df.index.levels
dates, tubes = elts_index[0], elts_index[1]

# getting null measure date
date_0 = '06.03.2020' # get_zero_measure_date(dates, start_date) # UMC2: QUI HO CAMBIATO QUALCOSA

nullmessungen_A = {}
nullmessungen_B = {}

# setting relative displacements columns
# A priori la df sera toujours à deux tubes I1 et I2 mais permet de prevenir les cp
for tube in tubes:
  nullmessungen_A[tube] = numpy.array(df.loc[idx[date_0, tube], ('A','defo')])
  nullmessungen_B[tube] = numpy.array(df.loc[idx[date_0, tube], ('B','defo')])

for date in dates:
  for tube in tubes:
    df.loc[idx[date, tube], ('A','defo relat')] = numpy.array(df.loc[idx[date, tube], ('A','defo')]) - nullmessungen_A[tube]
    df.loc[idx[date, tube], ('B','defo relat')] = numpy.array(df.loc[idx[date, tube], ('B','defo')]) - nullmessungen_B[tube]

# Remove measures before null measure
df = df.truncate(before=date_0)


#return df